In [1]:
## 1.a All observation has the same intial weight as 1/(number of observation) which is 1/100

## 1.b Because the observation is misclassified by the first learner, the new weight (second round)
#      will be larger than the initial weight. Subsequently, the next learner will focus on this
#      observation to find way to correctly classify it.

In [2]:
## 2. AdaBoost.M1 is an ensemble learning algorithy be cause it builds and uses more than 1 learner.
#     The framework sequentially build many learners to correctly classify the most number of
#     observations, until reaching a preset number of learners. Finally, the prediction is a
#     combined result of all learners built.

In [3]:
## 3. Observation      Old Weight      Correct?      New Weight
#         1               0.16         Correct
#         2               0.64         Correct
#         3               0.08         Incorrect
#         4               0.12         Incorrect
#     
#     epsilon = (0.08 + 0.12) / (0.16 + 0.64 + 0.08 + 0.12) = 0.2    < 0.5
#     alpha = log[(1 - 0.2) / 0.2] = 1.386
#     eta = 0.1
#
#     new weight 1 = 0.16 * e^(0.1 * 1.386 * (-1)) = 0.139
#     new weight 2 = 0.64 * e^(0.1 * 1.386 * (-1)) = 0.557
#     new weight 3 = 0.08 * e^(0.1 * 1.386 * 1) = 0.092   
#     new weight 4 = 0.12 * e^(0.1 * 1.386 * 1) = 0.138   

In [4]:
## 4. If an AdaBoost ensemble underfits the training dataset, the first thing that I would do is
#     to tweak the learning rate. Lower learning rate should help adapt to the dataset better.
#     Lower learning rate with a small number of learners could lead to underfitting as well, so
#     that would be the next thing I will check.

In [5]:
## 5. c

In [6]:
## 6.a
import boto3
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score
from sklearn.svm import SVC

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'danhtran358-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [7]:
## Defining input and target variables
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

In [8]:
## function to get recall from random forest model
def get_recall_rf(X_train, X_test, Y_train, Y_test):
    ## Random forest model
    md_rf = RandomForestClassifier(max_depth = 3, n_estimators = 500).fit(X_train, Y_train)
    
    ## Predict and classfify
    pred_rf = md_rf.predict_proba(X_test)[:,1]
    pred_rf = np.where(pred_rf < 0.1, 0, 1)
    
    ## Calculate recall score and return
    return recall_score(Y_test, pred_rf)

In [9]:
## function to get recall from AdaBoost model with Decision tree
def get_recall_ada_tree(X_train, X_test, Y_train, Y_test):
    ## Adaboost model with Decision Tree leaners
    md_ada_tree = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)
    
    ## Predict and classfify
    pred_ada_tree = md_ada_tree.predict_proba(X_test)[:,1]
    pred_ada_tree = np.where(pred_ada_tree < 0.1, 0, 1)
    
    ## Calculate recall score and return
    return recall_score(Y_test, pred_ada_tree)

In [10]:
## function to get recall from AdaBoost model with SVC
def get_recall_ada_svc(X_train, X_test, Y_train, Y_test):
    ## Adaboost model with Support Vector Machine leaners
    md_ada_svc = AdaBoostClassifier(base_estimator = SVC(kernel = 'rbf', probability = True), n_estimators = 50).fit(X_train, Y_train)
    
    ## Predict and classfify
    pred_ada_svc = md_ada_svc.predict_proba(X_test)[:,1]
    pred_ada_svc = np.where(pred_ada_svc < 0.1, 0, 1)
    
    ## Calculate recall score and return
    return recall_score(Y_test, pred_ada_svc)

In [11]:
## this a workaround because I lost kernel while running the loops
## file to store recall scores in S3
recall_file = bucket.Object('recall_scores.csv')
        

## function to write list of recalls to csv file in S3 bucket
def write_recalls(recall_list1, recall_list2, recall_list3):
    ## header
    content = 'RF,ADA Tree,ADA SVC'

    ## add content from the lists of recall scores
    for i in range(len(recall_list1)):
        content = content + '\n' + str(recall_list1[i]) + ',' + str(recall_list2[i]) + ',' + str(recall_list3[i])

    ## store as new csv file
    recall_file.put(Body = content)
    

## function to read list of recalls stored
def read_recall_scores():
    try:
        recall_file.load()
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            ## no score yet
            return [],[],[]
    else:
        ## return the lists of scores already stored
        recall_scores = pd.read_csv(recall_file.get().get('Body'))
        return list(recall_scores['RF']),list(recall_scores['ADA Tree']),list(recall_scores['ADA SVC'])

In [14]:
## retrieve the list of scores already stored
rf_recall, ada_tree_recall, ada_svc_recall = read_recall_scores()
scaler = MinMaxScaler()


for i in range(100 - len(rf_recall)):
    ## Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

    ## Scale features to range 0-1
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    ## store new recall scores to lists
    rf_recall.append(get_recall_rf(X_train, X_test, Y_train, Y_test))
    ada_tree_recall.append(get_recall_ada_tree(X_train, X_test, Y_train, Y_test))
    ada_svc_recall.append(get_recall_ada_svc(X_train, X_test, Y_train, Y_test))
    
    ## store the lists of recall scores to csv file in s3
    write_recalls(rf_recall, ada_tree_recall, ada_svc_recall)
    

## print the average recall scores
final_recall_scores = pd.read_csv(recall_file.get().get('Body'))
print('Average recall score of Random Forest:', np.mean(list(final_recall_scores['RF'])))
print('Average recall score of AdaBoost Tree:', np.mean(list(final_recall_scores['ADA Tree'])))
print('Average recall score of AdaBoost SVC: ', np.mean(list(final_recall_scores['ADA SVC'])))

Average recall score of Random Forest: 0.9335714285571428
Average recall score of AdaBoost Tree: 1.0
Average recall score of AdaBoost SVC:  1.0


In [15]:
## Based on recall score alone, I would choose either one of the 2 AdaBoost models to predict